In [1]:
import numpy  as np
import pandas as pd
import requests
import csv
import sys
jax_url = 'http://www.informatics.jax.org/downloads/reports/MGI_Geno_DiseaseDO.rpt'

In [2]:
#download = requests.get(jax_url) 

In [58]:

def parse_url(url,columns):
    df = pd.DataFrame()

    with requests.Session() as s:
        download = s.get(url)

        decoded_content = download.content.decode('utf-8')

        cr = csv.reader(decoded_content.splitlines(), delimiter='\t')
        my_list = list(cr)
        
        for i,row in enumerate(my_list,start=0):
            if i is 0: 
                df = pd.DataFrame(columns=columns)
                df.loc[i] = row
            else:
                df.loc[i] = row
    return df

cols=['Allelic Composition','Allele Symbol(s)','Allele ID(s)',
      'Genetic Background','Mammalian Phenotype ID','PubMed ID',
      'MGI Marker Accession ID','DOID']

## Mouse/Human ortholog with MP annotation

In [4]:
url_orth = 'http://www.informatics.jax.org/downloads/reports/HMD_HumanPhenotype.rpt'

In [5]:
columns = ['Human Marker Symbol','Human Entrez Gene ID','HomoloGene ID',
           'HGNC Association?','Mouse Marker Symbol','MGI Marker Accession ID',
           'High-level Mammalian Phenotype ID']

In [11]:
df = pd.DataFrame()

with requests.Session() as s:
    download = s.get(url_orth)
    columns = ['Human Marker Symbol','Human Entrez Gene ID','HomoloGene ID',
           'HGNC Association?','Mouse Marker Symbol','MGI Marker Accession ID',
           'High-level Mammalian Phenotype ID','']

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter='\t')
    my_list = list(cr)

    for i,row in enumerate(my_list,start=0):
        if i is 0: 
            df = pd.DataFrame(columns=columns)
            df.loc[i] = row
        else:
            df.loc[i] = row

In [43]:
df_filt = df[df['High-level Mammalian Phenotype ID'] != '']
df_filt = df_filt.reset_index(drop=True)

In [44]:
mp_list = []

for i in range(0,len(df_filt['High-level Mammalian Phenotype ID'])):
    stripped_mps  =[k.strip() for k in df_filt['High-level Mammalian Phenotype ID'][i].split(',')]
    mp_list.append(stripped_mps)

In [49]:
MG_MPO = pd.DataFrame(columns=['Gene','MPO'])
gene_list = []
mpo_list = []

for i,row in enumerate(mp_list):
    #if row[0] is '':
    current_gene = df_filt['Mouse Marker Symbol'][i]
    for mp_term in row:
        #        MG_MPO['Gene'][i] = current_gene
        #        MG_MPO['MPO'][i] = mp_term
        gene_list.append(current_gene)
        mpo_list.append(mp_term)
    

In [71]:
MG_MPO = pd.DataFrame(np.transpose([gene_list,mpo_list]),columns=['Gene','MPO'])
MG_MPO['Gene'].unique().shape
MG_MPO['MPO'].unique().shape

(27,)

## Mouse Genotypes with both Phenotype and Disease Annotations

In [54]:
url2 = 'http://www.informatics.jax.org/downloads/reports/MGI_Geno_DiseaseDO.rpt'


In [62]:
cols = ['Allelic Composition', 'Allele symbol(s)', 'Allele ID(s)', 
'Genetic background', 'MP ID', 'PubMed ID', 'MGI Marker ID', 'DO ID','OMIM']

In [63]:
df2  = parse_url(url2,columns=cols)

In [64]:
df2

,Allelic Composition,Allele symbol(s),Allele ID(s),Genetic background,MP ID,PubMed ID,MGI Marker ID,DO ID,
0,Ednra<tm1Ywa>/Ednra<tm1Ywa>,Ednra<tm1Ywa>,MGI:1857473,129S/SvEv-Ednra<tm1Ywa>,MP:0002127,9449664,MGI:105923,DOID:12583,OMIM:192430
1,Ednra<tm1Ywa>/Ednra<tm1Ywa>,Ednra<tm1Ywa>,MGI:1857473,129S/SvEv-Ednra<tm1Ywa>,MP:0000452,9449664,MGI:105923,DOID:12583,OMIM:192430
2,Ednra<tm1Ywa>/Ednra<tm1Ywa>,Ednra<tm1Ywa>,MGI:1857473,129S/SvEv-Ednra<tm1Ywa>,MP:0002108,9449664,MGI:105923,DOID:12583,OMIM:192430
3,Ednra<tm1Ywa>/Ednra<tm1Ywa>,Ednra<tm1Ywa>,MGI:1857473,129S/SvEv-Ednra<tm1Ywa>,MP:0000614,9449664,MGI:105923,DOID:12583,OMIM:192430
4,Ednra<tm1Ywa>/Ednra<tm1Ywa>,Ednra<tm1Ywa>,MGI:1857473,129S/SvEv-Ednra<tm1Ywa>,MP:0001823,9449664,MGI:105923,DOID:12583,OMIM:192430
...,...,...,...,...,...,...,...,...,...
36589,Sdr9c7<em1Maak>/Sdr9c7<em1Maak>,Sdr9c7<em1Maak>,MGI:6469363,involves: C57BL/6J,MP:0001222,31671075,MGI:1917311,DOID:0080257,OMIM:617574
36590,Sdr9c7<em1Maak>/Sdr9c7<em1Maak>,Sdr9c7<em1Maak>,MGI:6469363,involves: C57BL/6J,MP:0002796,31671075,MGI:1917311,DOID:0080257,OMIM:617574
36591,Sdr9c7<em1Maak>/Sdr9c7<em1Maak>,Sdr9c7<em1Maak>,MGI:6469363,involves: C57BL/6J,MP:0003853,31671075,MGI:1917311,DOID:0080257,OMIM:617574
36592,Sdr9c7<em1Maak>/Sdr9c7<em1Maak>,Sdr9c7<em1Maak>,MGI:6469363,involves: C57BL/6J,MP:0009594,31671075,MGI:1917311,DOID:0080257,OMIM:617574


In [66]:
for i in df2.columns:
    v = df2[i].unique().shape
    print(f'Unique values in {i} col: {v}')

Unique values in Allelic Composition col: (2812,)
Unique values in Allele symbol(s) col: (2811,)
Unique values in Allele ID(s) col: (2560,)
Unique values in Genetic background col: (1431,)
Unique values in MP ID col: (5964,)
Unique values in PubMed ID col: (4154,)
Unique values in MGI Marker ID col: (1464,)
Unique values in DO ID col: (1165,)
Unique values in  col: (1397,)
